In [ ]:
import jupyros
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import Twist

rospy.init_node('basics_node')

# Publishers and Subscribers

In [ ]:
topic = '/hello_world'

jupyros.publish(topic, String)

In [ ]:
jupyros.subscribe(topic, String, lambda msg: print(msg))

In [ ]:
topic = '/twist_world'

jupyros.publish(topic, Twist)

In [ ]:
jupyros.subscribe(topic, Twist, lambda msg: print(msg))

# Clients and Services

In [ ]:
from rospy_tutorials.srv import AddTwoInts, AddTwoIntsResponse

def handle_add_two_ints(req):
    print(f"Returning {req.a} + {req.b} = {req.a + req.b}")
    return AddTwoIntsResponse(req.a + req.b)

srv_name = 'add_two_ints'

In [ ]:
%%thread_cell

srv = rospy.Service(srv_name, AddTwoInts, handle_add_two_ints)

In [ ]:
jupyros.client(srv_name, AddTwoInts)

**Note:** Check log

# Turtlesim tf2

In [ ]:
from turtlesim.msg import Pose
from jupyros.turtle_sim import TurtleSim
import math

In [ ]:
turtlesim = TurtleSim()

# The canvas default size is 1600 x 1600 starting from top-left
turtlesim.spawn(name="turtle2", pose={"x": 630,
                                      "y": 1260,
                                      "theta": math.radians(90)})

turtlesim.turtles["turtle2"].path_color = "#32292F"

display(turtlesim.canvas)

In [ ]:
topic1 = '/turtle1/pose'
topic2 = '/turtle2/pose'
run = True

new_poses = {'turtle1': turtlesim.turtles['turtle1'].pose,
             'turtle2': turtlesim.turtles['turtle2'].pose}

def convert_xy_units(x, y):
    # Based on the dimensions of window controlled by teleop keyboard
    convert_factor = 11.08
    x_convert = x / convert_factor * turtlesim.canvas.width
    y_convert = (convert_factor - y) / convert_factor * turtlesim.canvas.height
    
    return x_convert, y_convert

def callback_move_turtle(msg, turtle_name):
    x, y = convert_xy_units(msg.x, msg.y)
    pose = {"x": x, "y": y, "theta": msg.theta}
    new_poses[turtle_name] = {"x": x, "y": y, "theta": msg.theta}

In [ ]:
%%thread_cell

rate = rospy.Rate(30) # 10 Hz
while run:
    turtlesim.move_turtles(new_poses)
    rate.sleep()
    
print('Done')    

In [ ]:
jupyros.subscribe(topic1, Pose, lambda msg: callback_move_turtle(msg, turtle_name="turtle1"))

In [ ]:
jupyros.subscribe(topic2, Pose, lambda msg: callback_move_turtle(msg, turtle_name="turtle2"))

- Run turtle.launch file
- [Optional] run turtle_teleop_key

In [ ]:
run = False